In [91]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from itertools import combinations

In [92]:
apple_df = pd.read_csv('./AAPL_stock_data.csv')
snapchat_df = pd.read_csv('./SNA_stock_data.csv')
twitter_df = pd.read_csv('./TWTR_stock_data.csv')

In [93]:
apple_df['stock_name'] = 'Apple'
snapchat_df['stock_name'] = 'Snapchat'
twitter_df['stock_name'] = 'Twitter'

In [94]:
combined_df = pd.concat([apple_df, snapchat_df, twitter_df], ignore_index=True)

In [95]:
basket = (combined_df.groupby(['date', 'stock_name'])['stock_name']
          .count().unstack().reset_index().fillna(0)
          .set_index('date'))

In [96]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [97]:
basket_sets = basket.applymap(encode_units)

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

/opt/homebrew/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [98]:
print(frequent_itemsets)

    support             itemsets
0  0.500000              (Apple)
1  0.500000           (Snapchat)
2  0.187676            (Twitter)
3  0.187676  (Twitter, Snapchat)


In [99]:
combined_df.to_csv('combined_stock_data.csv', index=False)

In [100]:
print(rules)

  antecedents consequents  antecedent support  consequent support   support  \
0   (Twitter)  (Snapchat)            0.187676            0.500000  0.187676   
1  (Snapchat)   (Twitter)            0.500000            0.187676  0.187676   

   confidence  lift  leverage  conviction  zhangs_metric  
0    1.000000   2.0  0.093838         inf       0.615518  
1    0.375353   2.0  0.093838    1.300452       1.000000  
